In [1]:
import os
import tensorflow as tf
from tensorflow.keras import Model
from tensorflow.keras import layers
import tensorflow.keras as keras
from PIL import Image
import numpy as np

In [2]:
src_dir = "../datasets/ori/"
train_dir = "../datasets/train/"
val_dir = "../datasets/validation/"
test_dir = "../datasets/test/"
categories = ['igneous/', 'metamorphic/', 'sedimentary/']
for category in categories:
    file_list = os.listdir(src_dir+category)
    for n, i in enumerate(file_list[:200]):
        img = Image.open(src_dir+category+i)
        img_resize = img.resize((150, 150))
        img_resize.save(train_dir+category[:-1]+"_"+str(n)+".jpg", "JPEG")
    for n, i in enumerate(file_list[200:300]):
        img = Image.open(src_dir+category+i)
        img_resize = img.resize((150, 150))
        img_resize.save(val_dir+category[:-1]+"_"+str(n)+".jpg", "JPEG")
    for n, i in enumerate(file_list[300:400]):
        img = Image.open(src_dir+category+i)
        img_resize = img.resize((150, 150))
        img_resize.save(test_dir+category[:-1]+"_"+str(n)+".jpg", "JPEG")

In [3]:
train_data = np.empty(shape=(1, 150, 150, 3), dtype=np.float)
for i in os.listdir(train_dir):
    img = Image.open(train_dir+i)
    train_data = np.append(train_data, [np.asarray(img)], axis=0)
train_data = np.delete(train_data, [0,0], axis=0)
train_data = train_data / 255.0

In [4]:
a = [0 for i in range(200)] + [1 for j in range(200)] + [2 for k in range(200)]
train_labels = np.array(a)

In [5]:
s = np.arange(600)
np.random.shuffle(s)
train_data = train_data[s]
train_labels = train_labels[s]
print(train_data.shape, train_labels.shape)

(600, 150, 150, 3) (600,)


In [6]:
with tf.device('/CPU:0'):
    inputs = keras.Input(shape=(150, 150, 3,))
    flat = layers.Flatten()(inputs)
    fc1 = layers.Dense(64, activation='relu')(flat)
    fc2 = layers.Dense(128, activation='relu')(fc1)
    fc3 = layers.Dense(256, activation='relu')(fc2)
    output = layers.Dense(3, activation='softmax')(fc3)
    model = Model(inputs=inputs, outputs=output, name="FC_Model")
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['acc'])
model.summary()

Model: "FC_Model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 150, 150, 3)]     0         
_________________________________________________________________
flatten (Flatten)            (None, 67500)             0         
_________________________________________________________________
dense (Dense)                (None, 64)                4320064   
_________________________________________________________________
dense_1 (Dense)              (None, 128)               8320      
_________________________________________________________________
dense_2 (Dense)              (None, 256)               33024     
_________________________________________________________________
dense_3 (Dense)              (None, 3)                 771       
Total params: 4,362,179
Trainable params: 4,362,179
Non-trainable params: 0
________________________________________________

In [7]:
with tf.device('/CPU:0'):
    model.fit(train_data, train_labels, epochs=100)

Epoch 1/100
19/19 [==============================] - 1s 13ms/step - loss: 4.0098 - acc: 0.3400
Epoch 2/100
19/19 [==============================] - 0s 13ms/step - loss: 1.5895 - acc: 0.3433
Epoch 3/100
19/19 [==============================] - 0s 14ms/step - loss: 1.1428 - acc: 0.3333
Epoch 4/100
19/19 [==============================] - 0s 14ms/step - loss: 1.2519 - acc: 0.3783
Epoch 5/100
19/19 [==============================] - 0s 15ms/step - loss: 1.1456 - acc: 0.3467
Epoch 6/100
19/19 [==============================] - 0s 13ms/step - loss: 1.1317 - acc: 0.3683
Epoch 7/100
19/19 [==============================] - 0s 14ms/step - loss: 1.0652 - acc: 0.4083
Epoch 8/100
19/19 [==============================] - 0s 14ms/step - loss: 1.0848 - acc: 0.4000
Epoch 9/100
19/19 [==============================] - 0s 13ms/step - loss: 1.1193 - acc: 0.3517
Epoch 10/100
19/19 [==============================] - 0s 13ms/step - loss: 1.1009 - acc: 0.3333
Epoch 11/100
19/19 [=============================

In [11]:
with tf.device('/CPU:0'):
    inputs = keras.Input(shape=(150, 150, 3,))
    conv1 = layers.Conv2D(64, (3, 3), activation='relu')(inputs)
    pool1 = layers.MaxPool2D((2, 2))(conv1)
    drop1 = layers.Dropout(0.4)(pool1)
    conv2 = layers.Conv2D(128, (3, 3), activation='relu')(drop1)
    pool2 = layers.MaxPool2D((2, 2))(conv2)
    drop2 = layers.Dropout(0.4)(pool2)
    conv3 = layers.Conv2D(256, (3, 3), activation='relu')(drop2)
    flat = layers.Flatten()(conv3)
    output = layers.Dense(3, activation='softmax')(flat)
    model = Model(inputs=inputs, outputs=output, name="CNN")
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['acc'])
model.summary()

Model: "CNN"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 150, 150, 3)]     0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 148, 148, 64)      1792      
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 74, 74, 64)        0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 74, 74, 64)        0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 72, 72, 128)       73856     
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 36, 36, 128)       0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 36, 36, 128)       0       

In [12]:
model.fit(train_data, train_labels, epochs=30, validation_split=0.2)

Epoch 1/30
15/15 [==============================] - 5s 254ms/step - loss: 1.9967 - acc: 0.3458 - val_loss: 1.0982 - val_acc: 0.4083
Epoch 2/30
15/15 [==============================] - 4s 243ms/step - loss: 1.1003 - acc: 0.3458 - val_loss: 1.0966 - val_acc: 0.3417
Epoch 3/30
15/15 [==============================] - 4s 242ms/step - loss: 1.0946 - acc: 0.3812 - val_loss: 1.0841 - val_acc: 0.3417
Epoch 4/30
15/15 [==============================] - 4s 242ms/step - loss: 1.0853 - acc: 0.3792 - val_loss: 1.0456 - val_acc: 0.4917
Epoch 5/30
15/15 [==============================] - 4s 242ms/step - loss: 1.0607 - acc: 0.4667 - val_loss: 1.0159 - val_acc: 0.3667
Epoch 6/30
15/15 [==============================] - 4s 242ms/step - loss: 1.0538 - acc: 0.3917 - val_loss: 0.9887 - val_acc: 0.5750
Epoch 7/30
15/15 [==============================] - 4s 243ms/step - loss: 0.9776 - acc: 0.5104 - val_loss: 0.9659 - val_acc: 0.5250
Epoch 8/30
15/15 [==============================] - 4s 242ms/step - loss: 0.